<a href="https://colab.research.google.com/github/dinithprimal/TensorFlow_HandsOn/blob/main/Ex07_Tensorflow_2x.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Natural Language Processing

## Bag of Words

In [1]:
vocab = {} # maps word to integer representing it
word_encoding = 1
def bag_of_words(text):
  global word_encoding

  words = text.lower().split(" ") # create a list of all of the words in the text
  bag = {} # store all of the encodings and their frequency

  for word in words:
    if word in vocab:
      encoding = vocab[word]
    else:
      vocab[word] = word_encoding
      encoding = word_encoding
      word_encoding += 1

    if encoding in bag:
      bag[encoding] += 1
    else:
      bag[encoding] = 1

  return bag

text = "this is a test to see if this test will work isis test a test"
bag = bag_of_words(text)
print(bag)
print(vocab)

{1: 2, 2: 1, 3: 2, 4: 4, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1, 10: 1}
{'this': 1, 'is': 2, 'a': 3, 'test': 4, 'to': 5, 'see': 6, 'if': 7, 'will': 8, 'work': 9, 'isis': 10}


## Sentiment Analysis

### Movie Review Dataset

In [2]:
%tensorflow_version 2.x
from keras.datasets import imdb
from keras.preprocessing import sequence
import tensorflow as tf
import os
import numpy as np

VOCAB_SIZE = 88584

MAXLEN = 250
BATCH_SIZE = 64

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = VOCAB_SIZE)

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
17464789/17464789 [==============================] - 2s 0us/step


In [3]:
# Let's look at one review
train_data[0]

[1,
 14,
 22,
 16,
 43,
 530,
 973,
 1622,
 1385,
 65,
 458,
 4468,
 66,
 3941,
 4,
 173,
 36,
 256,
 5,
 25,
 100,
 43,
 838,
 112,
 50,
 670,
 22665,
 9,
 35,
 480,
 284,
 5,
 150,
 4,
 172,
 112,
 167,
 21631,
 336,
 385,
 39,
 4,
 172,
 4536,
 1111,
 17,
 546,
 38,
 13,
 447,
 4,
 192,
 50,
 16,
 6,
 147,
 2025,
 19,
 14,
 22,
 4,
 1920,
 4613,
 469,
 4,
 22,
 71,
 87,
 12,
 16,
 43,
 530,
 38,
 76,
 15,
 13,
 1247,
 4,
 22,
 17,
 515,
 17,
 12,
 16,
 626,
 18,
 19193,
 5,
 62,
 386,
 12,
 8,
 316,
 8,
 106,
 5,
 4,
 2223,
 5244,
 16,
 480,
 66,
 3785,
 33,
 4,
 130,
 12,
 16,
 38,
 619,
 5,
 25,
 124,
 51,
 36,
 135,
 48,
 25,
 1415,
 33,
 6,
 22,
 12,
 215,
 28,
 77,
 52,
 5,
 14,
 407,
 16,
 82,
 10311,
 8,
 4,
 107,
 117,
 5952,
 15,
 256,
 4,
 31050,
 7,
 3766,
 5,
 723,
 36,
 71,
 43,
 530,
 476,
 26,
 400,
 317,
 46,
 7,
 4,
 12118,
 1029,
 13,
 104,
 88,
 4,
 381,
 15,
 297,
 98,
 32,
 2071,
 56,
 26,
 141,
 6,
 194,
 7486,
 18,
 4,
 226,
 22,
 21,
 134,
 476,
 26,
 480,
 5

### More Preprocessing

In [4]:
train_data = sequence.pad_sequences(train_data, MAXLEN)
test_data = sequence.pad_sequences(test_data, MAXLEN)

### Creating the Model

In [5]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          2834688   
                                                                 
 lstm (LSTM)                 (None, 32)                8320      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 2843041 (10.85 MB)
Trainable params: 2843041 (10.85 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Training

In [7]:
model.compile(
    loss='binary_crossentropy',
    optimizer='rmsprop',
    metrics=['acc']
)

history = model.fit(
    train_data,
    train_labels,
    epochs=10,
    validation_split=0.2
)

Epoch 1/10
625/625 [==============================] - 53s 80ms/step - loss: 0.4515 - acc: 0.7803 - val_loss: 0.3316 - val_acc: 0.8648
Epoch 2/10
625/625 [==============================] - 22s 35ms/step - loss: 0.2614 - acc: 0.8986 - val_loss: 0.2842 - val_acc: 0.8846
Epoch 3/10
625/625 [==============================] - 15s 24ms/step - loss: 0.2008 - acc: 0.9279 - val_loss: 0.3383 - val_acc: 0.8560
Epoch 4/10
625/625 [==============================] - 12s 19ms/step - loss: 0.1630 - acc: 0.9426 - val_loss: 0.2999 - val_acc: 0.8732
Epoch 5/10
625/625 [==============================] - 10s 16ms/step - loss: 0.1354 - acc: 0.9537 - val_loss: 0.3493 - val_acc: 0.8668
Epoch 6/10
625/625 [==============================] - 9s 15ms/step - loss: 0.1126 - acc: 0.9635 - val_loss: 0.3189 - val_acc: 0.8776
Epoch 7/10
625/625 [==============================] - 10s 16ms/step - loss: 0.0932 - acc: 0.9705 - val_loss: 0.3433 - val_acc: 0.8852
Epoch 8/10
625/625 [==============================] - 10s 16ms/

In [8]:
results = model.evaluate(test_data, test_labels)
print(results)

782/782 [==============================] - 4s 6ms/step - loss: 0.4836 - acc: 0.8630
[0.48362791538238525, 0.8629999756813049]


### Making Pridictions

In [9]:
word_index = imdb.get_word_index()

def encode_text(text):
  tokens = tf.keras.preprocessing.text.text_to_word_sequence(text)
  tokens = [word_index[word] if word in word_index else 0 for word in tokens]
  return sequence.pad_sequences([tokens], MAXLEN)[0]

text = 'that movie was just amazing, so amazing'
encoded = encode_text(text)
print(encoded)

1641221/1641221 [==============================] - 1s 1us/step
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 

In [10]:
# While were at it let's make a decode function

reverse_word_index = {value: key for (key, value) in word_index.items()}

def decode_integers(integers):
  PAD = 0
  text = ""
  for num in integers:
    if num != PAD:
      text += reverse_word_index[num] + " "

  return text[:-1]

print(decode_integers(encoded))

that movie was just amazing so amazing


In [11]:
# Now time to make a prediction

def predict(text):
  encoded_text = encode_text(text)
  pred = np.zeros((1, 250))
  pred[0] = encoded_text
  result = model.predict(pred)
  print(result[0])

positive_review = "That movie was so awesome! I really loved it and would watch it again becouse it was amazingly greate"
predict(positive_review)

negative_review = "that movie sucked. I hated it and wouldn't watch it again. Was one of the wrost things I've ever watched"
predict(negative_review)

1/1 [==============================] - 0s 469ms/step
[0.56315136]
1/1 [==============================] - 0s 20ms/step
[0.13009317]
